In [1]:
import pandas as pd
import numpy as np
import json, csv, datetime

In [2]:
#input

file_path = '/home/adrian/Dokumente/real_data/IDAWEB/idaweb3/order50466/data3.csv'

#output
output_path = '/home/adrian/Dokumente/real_data/idaweb_out/'

test_file_name = output_path + 'test.csv'
train_file_name = output_path + 'train.csv'

In [3]:
def load_data_frame(file_path):
    df = pd.read_csv(file_path,
                     sep=';',
                     parse_dates=['time'],
                     infer_datetime_format=True,
                     index_col='time',
                     float_precision='high')
    return df

In [4]:
def init_file(outputfile):
    with open(outputfile, 'w') as file_output:
        writer = csv.writer(file_output, delimiter=';')
        header = ['ts_name', 'time', 'unit', 'value', 'class']
        writer.writerow(header)
        
def append_to_file(df, ts_name, unit, outputfile):
    with open(outputfile, 'a') as file_output:
        writer = csv.writer(file_output, delimiter=';')
        for index, value in df.iteritems():
            row = [ts_name, index.isoformat(), unit, value, 0]
            writer.writerow(row)

In [20]:
ts_names_14 = ['REH', 'KLO', 'SHA', 'NABZUE', 'SMA', 'WYN', 'WAE', 'SAG', 'SRS', 'THU', 'VAD', 'RUE', 'HAI', 'VLS']
ts_names_12 = ['REH', 'KLO', 'SHA', 'NABZUE', 'SMA', 'WYN', 'WAE', 'SAG', 'SRS', 'THU', 'VAD', 'RUE']
ts_names_10 = ['REH', 'KLO', 'SHA', 'NABZUE', 'SMA', 'WYN', 'WAE', 'SAG', 'SRS', 'THU']
ts_names_8 = ['REH', 'KLO', 'SHA', 'NABZUE', 'SMA','WYN', 'WAE', 'SAG']
ts_names_6 = ['REH', 'KLO', 'SHA', 'NABZUE', 'SMA','WYN']
ts_names_4 = ['REH', 'KLO', 'SHA', 'NABZUE']
ts_names_2 = ['REH', 'KLO']

In [14]:
df_read = load_data_frame(file_path)

In [15]:
df = df_read.pivot(columns='ts_name', values='value')

In [22]:
def generate_output(df, ts_names, path_addition, file_suffix, ts_length=1000, training_length=500):
    test_file_name = output_path + path_addition + 'test_' + file_suffix + '.csv'
    train_file_name = output_path + path_addition + 'train_' + file_suffix + '.csv'
    init_file(test_file_name)
    init_file(train_file_name)
    
    for ts_name in ts_names:
        df_set = df[ts_name]
        df_test = df_set.iloc[0:1000]
        df_train = df_set.iloc[1000:2000]
    
        append_to_file(df_test, ts_name, 'Celsius', test_file_name)
        append_to_file(df_train, ts_name, 'Celsius', train_file_name)

In [23]:
generate_output(df, ts_names_14, 'set1/', file_suffix='14', ts_length=1000, training_length=500)
generate_output(df, ts_names_12, 'set1/', file_suffix='12', ts_length=1000, training_length=500)
generate_output(df, ts_names_10, 'set1/', file_suffix='10', ts_length=1000, training_length=500)
generate_output(df, ts_names_8, 'set1/', file_suffix='8', ts_length=1000, training_length=500)
generate_output(df, ts_names_6, 'set1/', file_suffix='6', ts_length=1000, training_length=500)
generate_output(df, ts_names_4, 'set1/', file_suffix='4', ts_length=1000, training_length=500)
generate_output(df, ts_names_2, 'set1/', file_suffix='2', ts_length=1000, training_length=500)

In [57]:
#Correlation

df = df_read.pivot(columns='ts_name', values='value')
df_no_na = df.dropna(axis=1, how='any')
print(df_no_na)
df_corr = df_no_na.corr(method='pearson')
df_corr = df_corr.round(2)
df_corr.to_csv(output_path + 'set1/' +'corr.csv', index = True, header=True)

ts_name              DLSAA  DLSTO  DLSTU  DLWEI   HAI  KLO  MMWEG  NABZUE  \
time                                                                        
2017-04-01 00:00:00   10.1   12.1   10.8    5.5  13.1  7.9    6.6    13.1   
2017-04-01 00:10:00   10.2   11.7    9.5    6.4  13.3  9.0    6.7    12.6   
2017-04-01 00:20:00   10.0   11.5    9.2    6.3  12.6  8.8    5.7    12.5   
2017-04-01 00:30:00    9.2   11.2    8.6    4.1  12.6  8.2    6.7    13.0   
2017-04-01 00:40:00    9.0   11.1    8.8    5.3  12.8  7.8    7.6    12.8   
...                    ...    ...    ...    ...   ...  ...    ...     ...   
2017-04-30 23:10:00   13.2    5.0    6.3    6.2  10.0  5.8    3.8    10.4   
2017-04-30 23:20:00   13.4    4.6    6.7    6.1  10.1  4.0    3.1    10.4   
2017-04-30 23:30:00   13.2    4.3    7.5    5.6   9.9  4.1    3.8    10.5   
2017-04-30 23:40:00   12.8    4.5    6.8    5.0  10.2  1.8    2.7    11.3   
2017-04-30 23:50:00   12.6    4.3    6.9    5.3  10.4  3.7    2.6    11.1   

In [7]:
def create_set(path, files, file_set_names, file_suffix):
    test_file_name = output_path + path + 'test_' + file_suffix + '.csv'
    train_file_name = output_path + path + 'train_' + file_suffix + '.csv'
    init_file(test_file_name)
    init_file(train_file_name)
    
    for entry in files:
        ts_name = entry['name']
        if ts_name in file_set_names:
            file_path = entry['file_path']
            df = load_data_frame(ts_name, file_path)
            df = time_index_to_dt(df)
            print(ts_name + "\t anomalies " 
                  + str(df.loc[df['class']==1].shape[0]) 
                  + '\t values ' + str(df.shape[0])
                  + '\t min ' + str(df.loc[df['class']==0, 'value'].min()) 
                  + '\t max ' + str(df.loc[df['class']==0, 'value'].max()))
            df_test = df.iloc[0:710]
            print('test' + "\t anomalies " + str(df_test.loc[df_test['class']==1].shape[0]) + '\t values ' + str(df_test.shape[0]))

            df_train = df.iloc[710:1421]
            print('train' + "\t anomalies " + str(df_train.loc[df_train['class']==1].shape[0]) + '\t values ' + str(df_train.shape[0]))
            print('\n')
            append_to_file(df_test, test_file_name)
            append_to_file(df_train, train_file_name)